In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        getOrCreate()

22/02/03 16:48:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [12]:
df = spark.read.option("header", "true").csv("data/ddos/ddos_imbalanced/*.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Flow ID: string (nullable = true)
 |-- Src IP: string (nullable = true)
 |-- Src Port: string (nullable = true)
 |-- Dst IP: string (nullable = true)
 |-- Dst Port: string (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: string (nullable = true)
 |-- Tot Fwd Pkts: string (nullable = true)
 |-- Tot Bwd Pkts: string (nullable = true)
 |-- TotLen Fwd Pkts: string (nullable = true)
 |-- TotLen Bwd Pkts: string (nullable = true)
 |-- Fwd Pkt Len Max: string (nullable = true)
 |-- Fwd Pkt Len Min: string (nullable = true)
 |-- Fwd Pkt Len Mean: string (nullable = true)
 |-- Fwd Pkt Len Std: string (nullable = true)
 |-- Bwd Pkt Len Max: string (nullable = true)
 |-- Bwd Pkt Len Min: string (nullable = true)
 |-- Bwd Pkt Len Mean: string (nullable = true)
 |-- Bwd Pkt Len Std: string (nullable = true)
 |-- Flow Byts/s: string (nullable = true)
 |-- Flow Pkts/s: string (nullable =

In [37]:
df = df.select("Src IP", "Dst Port", "Dst Ip", "Protocol", "_c0")
df.show()

+--------------+--------+-------------+--------+-------+
|        Src IP|Dst Port|       Dst Ip|Protocol|    _c0|
+--------------+--------+-------------+--------+-------+
| 18.219.193.20|      80| 172.31.69.25|       6|1739476|
|  172.31.69.28|   63287|   18.219.9.1|       6|1822666|
| 52.14.136.135|      80| 172.31.69.28|       6| 905739|
|18.216.200.189|      80| 172.31.69.28|       6|1143064|
|  172.31.69.28|   57459|18.218.55.126|       6|1934016|
|  172.31.69.28|   56276|   18.219.9.1|       6|1916750|
|  172.31.69.25|   40594|18.219.193.20|       6|2170484|
|  172.31.69.28|   53734|  18.219.5.43|       6|1891912|
| 18.219.193.20|      80| 172.31.69.25|       6| 262892|
| 18.219.193.20|      80| 172.31.69.25|       6| 170886|
| 18.219.193.20|      80| 172.31.69.25|       6| 847148|
|  172.31.69.25|   56310|18.219.193.20|       6|3142101|
|  172.31.69.25|   41060|18.219.193.20|       6|2964667|
|  172.31.69.25|   48258|18.219.193.20|       6|2845398|
| 18.219.193.20|      80| 172.3

In [38]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.sql.functions import col, avg
from pyspark.sql.types import StringType, IntegerType


#plot quantos serviços diferentes um IP é atacado
#dst_ip_port_distinct = df.distinct().groupBy("Dst IP").count().orderBy(col("count").desc())
#dst_ip_port.show()

#Quais IPs foram mais atacados:
dst_ips = df.groupBy("Dst IP").count().orderBy(col("count").desc())
dst_ips.show()

#Quais IPs foram mais spoofados? 
src_ips = df.groupBy("Src IP").count().orderBy(col("count").desc())
src_ips.show()

#Quais portas/serviços foram mais atacados 
dst_port = df.groupBy("Dst Port").count().orderBy(col("count").desc())
dst_port.show()

#Quais IPs/serviços foram mais atacados?
dst_ip_port = df.groupBy("Dst IP","Dst Port").count().orderBy(col("count").desc())
dst_ip_port.show()


#Media de duração de um fluxo de ataque: (min)
df = df.withColumn("_c0",col("_c0").cast(IntegerType()))
flow = df.select(avg(col('_c0'))/60000)
flow.show()


+---------------+--------+-------+
|         Dst IP|Dst Port|  count|
+---------------+--------+-------+
|     172.31.0.2|      53|1965938|
|   172.31.69.25|      80| 475283|
|169.254.169.254|      80| 439436|
|   172.31.69.28|      80| 250274|
|   192.168.10.3|      53| 118089|
|        8.0.6.4|       0|  75213|
|   192.168.10.1|      53|  53501|
|   178.255.83.1|      80|  39014|
|    72.21.91.29|      80|  32189|
|    23.52.91.27|      80|  29655|
|   172.31.69.25|      21|  21245|
| 72.167.239.239|      80|  19375|
|    224.0.0.252|    5355|  14848|
|   23.218.62.30|     443|  14448|
|    31.13.65.36|     443|  13182|
|     31.13.65.7|     443|  12980|
|  172.31.64.103|    3128|  12825|
| 216.58.211.162|     443|  10667|
|  216.58.198.66|     443|  10433|
|  13.89.190.129|     443|   9328|
+---------------+--------+-------+
only showing top 20 rows



+---------------+-------+
|         Dst IP|  count|
+---------------+-------+
|     172.31.0.2|1965938|
|   172.31.69.25| 496984|
|169.254.169.254| 439436|
|  18.219.193.20| 352981|
|   172.31.69.28| 255294|
|   192.168.10.3| 121753|
|        8.0.6.4|  75213|
|   192.168.10.1|  53503|
|   178.255.83.1|  39014|
|    72.21.91.29|  32190|
|  192.168.10.25|  31134|
|    23.52.91.27|  29655|
|   23.218.62.30|  21981|
| 72.167.239.239|  19375|
|  172.31.64.103|  19246|
| 18.218.229.235|  18757|
| 18.216.200.189|  18615|
|  18.218.55.126|  18568|
|  18.218.115.60|  18536|
|     18.219.9.1|  18488|
+---------------+-------+
only showing top 20 rows



+--------------+------+
|        Src IP| count|
+--------------+------+
|  172.31.69.25|353808|
| 18.219.193.20|348970|
|  172.31.69.28|194822|
|       8.6.0.1| 75213|
|  192.168.10.3| 54116|
| 192.168.10.25| 46393|
|  192.168.10.5| 39035|
|18.218.229.235| 37035|
|18.216.200.189| 36992|
|  18.218.11.51| 36770|
|    18.219.9.1| 36758|
|  18.216.24.42| 36721|
|   18.219.5.43| 36460|
| 52.14.136.135| 36402|
| 18.218.55.126| 36363|
|  18.219.32.43| 36356|
| 18.218.115.60| 36126|
|  212.92.116.6| 35115|
| 212.92.114.68| 32859|
|  37.46.246.28| 32847|
+--------------+------+
only showing top 20 rows



+--------+-------+
|Dst Port|  count|
+--------+-------+
|      53|2137540|
|      80|1472708|
|     443|1089062|
|    3389| 872249|
|     445| 344276|
|       0| 115562|
|      21|  22211|
|    5355|  14851|
|    3128|  12830|
|     123|  11973|
|     137|   8304|
|      67|   6499|
|      22|   3686|
|      23|   2835|
|     138|   1816|
|   27017|   1708|
|   49152|   1286|
|     139|   1004|
|     389|    972|
|      88|    964|
+--------+-------+
only showing top 20 rows



+---------------+--------+-------+
|         Dst IP|Dst Port|  count|
+---------------+--------+-------+
|     172.31.0.2|      53|1965938|
|   172.31.69.25|      80| 475283|
|169.254.169.254|      80| 439436|
|   172.31.69.28|      80| 250274|
|   192.168.10.3|      53| 118089|
|        8.0.6.4|       0|  75213|
|   192.168.10.1|      53|  53501|
|   178.255.83.1|      80|  39014|
|    72.21.91.29|      80|  32189|
|    23.52.91.27|      80|  29655|
|   172.31.69.25|      21|  21245|
| 72.167.239.239|      80|  19375|
|    224.0.0.252|    5355|  14848|
|   23.218.62.30|     443|  14448|
|    31.13.65.36|     443|  13182|
|     31.13.65.7|     443|  12980|
|  172.31.64.103|    3128|  12825|
| 216.58.211.162|     443|  10667|
|  216.58.198.66|     443|  10433|
|  13.89.190.129|     443|   9328|
+---------------+--------+-------+
only showing top 20 rows



+------------------+
|(avg(_c0) / 60000)|
+------------------+
|58.635676683186915|
+------------------+

